**Inorder to merge samples from different lanes Create a sample description file and annotate it**

```
(echo 'samplename,description'; for f in fastq*; do readlink -f $f | perl -pe 's/(.*?_(S[0-9]+)_.*)/\1,\2/'; done) > tomerge.csv 
```

**Submit a job to merge the files togeather**

```
#!/bin/sh
#SBATCH -p medium
#SBATCH -J bcbio_merge
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 1-00:00
#SBATCH --cpus-per-task=3
#SBATCH --mem=80G
#SBATCH --mail-type=END         # Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=ajitj_nirmal@dfci.harvard.edu   # Email to which notifications will be sent

module load bcbio/latest
bcbio_prepare_samples.py --out merged --csv tomerge.csv
```

**Prepare alignment.csv**
The tomerge-merged.csv produced by bcbio was edited to include meta data and alignment.csv was prepared.

**Download the reference genome**

```
mkdir reference
wget ftp://ftp.ensembl.org/pub/release-96/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
wget ftp://ftp.ensembl.org/pub/release-96/gtf/homo_sapiens/Homo_sapiens.GRCh38.96.gtf.gz
gunzip *
```

**Prepare the O2.yaml file**
```
details:
  - analysis: RNA-seq
    genome_build: hg38
    algorithm:
      transcriptome_fasta: /n/scratch2/ajit/FC_05641/reference/Homo_sapiens.GRCh38.cdna.all.fa
      transcriptome_gtf: /n/scratch2/ajit/FC_05641/reference/Homo_sapiens.GRCh38.96.gtf
      aligner: hisat2
      trim_reads: read_through
      adapters: [nextera, polya]
      strandedness: unstranded
      tools_on: [bcbiornaseq]
      bcbiornaseq:
          organism: homo sapiens
          interesting_groups: region
upload:
  dir: ../final
```

**Intiate bcBio**

```
module load bcbio/latest
unset PYTHONPATH
bcbio_nextgen.py -w template O2.yaml alignment.csv raw_files/
```

**Submit Job to O2**

```
cd alignment/work
vim submit_bcbio.sh

#!/bin/sh
#SBATCH -p medium
#SBATCH -J bcbio_O2              
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 5-00:00
#SBATCH --cpus-per-task=3
#SBATCH --mem=80G
#SBATCH --mail-type=END         # Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=ajitj_nirmal@dfci.harvard.edu   # Email to which notifications will be sent

export PATH=/n/app/bcbio/tools/bin:$PATH
bcbio_nextgen.py ../config/alignment.yaml \
    -n 24 -t ipython -s slurm -q medium -r t=5-00:00 --timeout 2000

sbatch submit_bcbio.sh
```